#Files needed
* rbfnn_v2.h5
* CNN_MNIST.h5
* adv_de_cnn_mnist_seed1
* adv_de_rbf_mnist_seed1
* subset

# Import Libraries

In [1]:
!pip install keras==2.2.4
!pip install tensorflow==1.13.1
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

     |████████████████████████████████| 317kB 2.8MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
     |████████████████████████████████| 92.5MB 57kB/s 
     |████████████████████████████████| 3.2MB 47.4MB/s 
     |████████████████████████████████| 368kB 45.6MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
  Cloning https://github.com/darecophoenixx/wordroid.sblo.jp to /tmp/pip-req-build-8vs_zqte
  Running command git clone -q https://github.com/darecophoenixx/wordroid.sblo.jp /tmp/pip-req-build-8vs_zqte
  Created wheel for wordroid.sblo.

In [2]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
import tensorflow as tf
from keras_ex.gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from scipy.stats import cauchy 
from scipy import stats
import operator
import random
from sklearn.metrics import accuracy_score
from skimage.measure import compare_ssim as ssim
import pickle
import cv2
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, Activation

#Set seeds
random.seed(0)
np.random.seed(0)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

# Get the data from pickle files

In [0]:

original = None
adversarial = None 
original_y = None
adversarial_y = None
dump = None
adversarial_rbf = None
adversarial_rbf_y  = None
adversarial_cnn = None
adversarial_cnn_y  = None

with open('subset', 'rb') as f:
    original = pickle.load(f)
    adversarial= pickle.load(f)
    original_y = pickle.load(f)
    adversarial_y = pickle.load(f)

with open('adv_de_rbf_mnist_seed1', 'rb') as f:
    adversarial_rbf = pickle.load(f)
    for i in range(3):
      dump = pickle.load(f)
    adversarial_rbf_y= pickle.load(f)

with open('adv_de_cnn_mnist_seed1', 'rb') as f:
    adversarial_cnn = pickle.load(f)
    for i in range(3):
      dump = pickle.load(f)
    adversarial_cnn_y= pickle.load(f)

In [4]:
original.shape

(895, 784)

In [5]:
adversarial.shape

(126, 784)

In [6]:
adversarial_rbf.shape

(126, 784)

# Calculate PSNR and SSIM and Euclidean

In [0]:
psnrs_rbf = []
psnrs_cnn = []
for i in range(adversarial.shape[0]):
  psnrs_rbf.append(cv2.PSNR(adversarial[i], adversarial_rbf[i]))
  psnrs_cnn.append(cv2.PSNR(adversarial[i], adversarial_cnn[i]))

In [8]:
#double check GeeksForGeeks psnr
print(np.around(np.mean(psnrs_rbf),3), "+-" ,np.around(np.std(psnrs_rbf),3),"(" ,np.around(np.min(psnrs_rbf),3), ",",
                                                                                        np.around(np.max(psnrs_rbf),3),")")
print(np.around(np.mean(psnrs_cnn),3), "+-" ,np.around(np.std(psnrs_cnn),3),"(" ,np.around(np.min(psnrs_cnn),3), ",",
                                                                                        np.around(np.max(psnrs_cnn),3),")")

19.242 +- 6.652 ( 10.9 , 29.642 )
13.641 +- 1.092 ( 11.117 , 15.559 )


In [9]:
ssims_rbf = []
ssims_cnn = []
for i in range(adversarial.shape[0]):
  ssims_rbf.append(ssim(adversarial[i].reshape(28,28), adversarial_rbf[i].reshape(28,28)))
  ssims_cnn.append(ssim(adversarial[i].reshape(28,28), adversarial_cnn[i].reshape(28,28)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  """


In [10]:
print(np.around(np.mean(ssims_rbf),3), "+-" ,np.around(np.std(ssims_rbf),3),"(" ,np.around(np.min(ssims_rbf),3), ",",
                                                                                        np.around(np.max(ssims_rbf),3),")")
print(np.around(np.mean(ssims_cnn),3), "+-" ,np.around(np.std(ssims_cnn),3),"(" ,np.around(np.min(ssims_cnn),3), ",",
                                                                                        np.around(np.max(ssims_cnn),3),")")

0.606 +- 0.155 ( 0.261 , 0.903 )
0.507 +- 0.112 ( 0.252 , 0.754 )


In [0]:
# https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
def euclidean(s):
    # pass s=a-b as argument to the function
    return np.linalg.norm(s)

In [0]:
eucs_rbf = []
eucs_cnn = []
for i in range(adversarial.shape[0]):
  eucs_rbf.append(euclidean(adversarial[i] - adversarial_rbf[i]))
  eucs_cnn.append(euclidean(adversarial[i] -  adversarial_cnn[i]))

In [13]:
print(np.around(np.mean(eucs_rbf),3), "+-" ,np.around(np.std(eucs_rbf),3),"(" ,np.around(np.min(eucs_rbf),3), ",",
                                                                                        np.around(np.max(eucs_rbf),3),")")
print(np.around(np.mean(eucs_cnn),3), "+-" ,np.around(np.std(eucs_cnn),3),"(" ,np.around(np.min(eucs_cnn),3), ",",
                                                                                        np.around(np.max(eucs_cnn),3),")")

6026.642 +- 315.534 ( 5135.876 , 6718.164 )
5593.703 +- 198.59 ( 4478.782 , 5952.044 )


# Prepare Date

In [0]:
#Normalize images
original_sc = original        /255.0
adversarial_sc = adversarial     /255.0 
adversarial_rbf_sc = adversarial_rbf /255.0
adversarial_cnn_sc = adversarial_cnn /255.0

In [0]:
#Prepare dims for CNN
cnn_original_sc = original_sc.reshape(original_sc.shape[0],28,28,1)
cnn_adversarial_sc = adversarial_sc.reshape(adversarial_sc.shape[0],28,28,1)
cnn_adversarial_rbf_sc= adversarial_rbf_sc.reshape(adversarial_rbf_sc.shape[0],28,28,1)
cnn_adversarial_cnn_sc= adversarial_cnn_sc.reshape(adversarial_cnn_sc.shape[0],28,28,1)

# Predict using RBFNN

In [16]:
num_lm0 = 100
num_lm = num_lm0 * 10
init_wgt = np.zeros(((1000, 784)))

inp = Input(shape=(28*28,), name='inp')
oup = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp)
oup = Dense(10, activation='softmax')(oup)
model = Model(inp, oup)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights("rbfnn_v2.h5")

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Get predictions
pred_original = model.predict(original_sc)
pred_adversarial = model.predict(adversarial_sc)
pred_adversarial_rbf = model.predict(adversarial_rbf_sc)
pred_adversarial_cnn = model.predict(adversarial_cnn_sc)

In [18]:
print("Accuracy on original: ",accuracy_score(original_y,np.argmax(pred_original,axis=1)))
print("Accuracy on adversarial: ",accuracy_score(adversarial_y,np.argmax(pred_adversarial,axis=1)))
print("Accuracy on adversarial_rbf: ",accuracy_score(adversarial_y,np.argmax(pred_adversarial_rbf,axis=1)))
print("Accuracy on adversarial_cnn: ",accuracy_score(adversarial_y,np.argmax(pred_adversarial_cnn,axis=1)))

Accuracy on original:  0.9363128491620112
Accuracy on adversarial:  0.9365079365079365
Accuracy on adversarial_rbf:  0.30158730158730157
Accuracy on adversarial_cnn:  0.4365079365079365


In [19]:
print("Accuracy on subset: ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(pred_original,pred_adversarial,axis=0),axis=1)))
print("Accuracy on subset with adversarial examples (RBF): ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(pred_original,pred_adversarial_rbf,axis=0),axis=1)))
print("Accuracy on subset with adversarial examples (CNN): ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(pred_original,pred_adversarial_cnn,axis=0),axis=1)))

Accuracy on subset:  0.9363369245837414
Accuracy on subset with adversarial examples (RBF):  0.8579823702252694
Accuracy on subset with adversarial examples (CNN):  0.8746327130264446


# Predict Using CNN

In [20]:
inp = Input(shape=(28,28,1))
conv1 = Conv2D(32, (3,3),activation='relu')(inp)
conv2 = Conv2D(32, (3,3),activation='relu')(conv1)
max_pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
drop = Dropout(0.25)(max_pool1)
flat =  Flatten()(drop)
dense1 = Dense(128,activation="relu")(flat)
drop = Dropout(0.5)(dense1)
predictions = Dense(10,activation="softmax")(drop) # softmax layer 
model = Model(inputs=[inp], outputs=[predictions])
model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.load_weights('CNN_MNIST.h5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#Get predictions
cnn_pred_original = model.predict(cnn_original_sc)
cnn_pred_adversarial = model.predict(cnn_adversarial_sc)
cnn_pred_adversarial_rbf = model.predict(cnn_adversarial_rbf_sc)
cnn_pred_adversarial_cnn = model.predict(cnn_adversarial_cnn_sc)

In [22]:
print("Accuracy on original: ",accuracy_score(original_y,np.argmax(cnn_pred_original,axis=1)))
print("Accuracy on adversarial: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial,axis=1)))
print("Accuracy on adversarial_rbf: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial_rbf,axis=1)))
print("Accuracy on adversarial_cnn: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial_cnn,axis=1)))

Accuracy on original:  0.9888268156424581
Accuracy on adversarial:  0.9920634920634921
Accuracy on adversarial_rbf:  0.9126984126984127
Accuracy on adversarial_cnn:  0.1349206349206349


In [23]:
print("Accuracy on subset: ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial,axis=0),axis=1)))
print("Accuracy on subset with adversarial examples (RBF): ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial_rbf,axis=0),axis=1)))
print("Accuracy on subset with adversarial examples (CNN): ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial_cnn,axis=0),axis=1)))

Accuracy on subset:  0.9892262487757101
Accuracy on subset with adversarial examples (RBF):  0.9794319294809011
Accuracy on subset with adversarial examples (CNN):  0.8834476003917727
